# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Initialization

load_dotenv()

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")
    
MODEL = "gpt-4o"
openai = OpenAI()

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
# MODEL = "llama3.2"
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


OpenAI API Key exists and begins sk-proj-


In [3]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [4]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7878

To create a public link, set `share=True` in `launch()`.


## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [5]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [6]:
get_ticket_price("Berlin")

Tool get_ticket_price called for Berlin


'$499'

In [7]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [8]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}]

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [9]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [12]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    print(tool_call)
    arguments = json.loads(tool_call.function.arguments)
    print(arguments)
    city = arguments.get('destination_city')
    print(f"Tool get_ticket_price called for {city}")
    price = get_ticket_price(city)
    print(f"Tool get_ticket_price returned {price}")
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [13]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7880

To create a public link, set `share=True` in `launch()`.


ChatCompletionMessageToolCall(id='call_DMlWoytkTCt7wB6VFUYb2Mxz', function=Function(arguments='{"destination_city":"Berlin"}', name='get_ticket_price'), type='function')
{'destination_city': 'Berlin'}
Tool get_ticket_price called for Berlin
Tool get_ticket_price called for Berlin
Tool get_ticket_price returned $499
ChatCompletionMessageToolCall(id='call_NhJzg5XeeNi96ZPFgljKmkGH', function=Function(arguments='{"destination_city":"New York"}', name='get_ticket_price'), type='function')
{'destination_city': 'New York'}
Tool get_ticket_price called for New York
Tool get_ticket_price called for New York
Tool get_ticket_price returned Unknown


In [ ]:
# Let's create some data simulating flights, seats, etc so we can simulate booking flights
# The data will be stored in lists of dicts

flights = [
    {"city": "London", "price": "$799", "seats": 100},
    {"city": "Paris", "price": "$899", "seats": 100},
    {"city": "Tokyo", "price": "$1400", "seats": 100},
    {"city": "Berlin", "price": "$499", "seats": 100},
]
#retrieve flight information
def get_flight(city):
    print(f"Tool get_flight called for {city}")
    for flight in flights:
        if flight["city"] == city:
            return flight
    return None # If the city is not found, return None 

flight_function = {
    "name": "get_flight",
    "description": "Get the flight details for the destination city. Call this whenever you need to know the flight details, for example when a customer asks 'What is the flight to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

tools.append({"type": "function", "function": flight_function})

def book_flight(flight):
    print(f"Tool book_flight called for {flight['city']}")
    if flight["seats"] > 0:
        flight["seats"] -= 1
        return f"Flight to {flight['city']} has been booked"
    else:
        return f"No seats available for {flight['city']}"
    return "There are no flights to that city"

book_flight_function = {
    "name": "book_flight",
    "description": "Book a flight to the destination city. Call this whenever a customer wants to book a flight",
    "parameters": {
        "type": "object",
        "properties": {
            "flight": {
                "type": "object",
                "description": "The flight details",
            },
        },
        "required": ["flight"],
        "additionalProperties": False
    }
}

tools.append({"type": "function", "function": book_flight_function})

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content